In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import eval_extraction
import openai
import extract
openai.api_key = open('/home/chansingh/.OPENAI_KEY').read().strip()

# Load the data

In [2]:
df = pd.read_pickle("../data/data_clean.pkl")[
    [
        "id",
        "paper___url",
        "paper___raw_text",
        "paper___year",
        "participants___male",
        "participants___female",
        "participants___total",
        "participants___white",
        "participants___black",
        "participants___latino",
        "participants___asian",
    ]
]

In [6]:
df.tail(n=300)

,id,paper___url,paper___raw_text,paper___year,participants___male,participants___female,participants___total,participants___white,participants___black,participants___latino,participants___asian
390,324,https://www.ncbi.nlm.nih.gov/pmc/articles/PMC1...,RESPIRATORY INFECTION\nDefining community acqu...,2003,550.0,518.0,1068.0,-1,-1,-1,-1
391,4012,https://www.ncbi.nlm.nih.gov/pubmed/12765673,Clinical Nutrition (2003) 22(3): 321–336\nr 20...,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
392,4003,https://www.ncbi.nlm.nih.gov/pubmed/12390332,"Clinical Endocrinology (2002) 57, 595–601\n© 2...",2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,3926,-1,Pediatric Appendicitis Score\nBy Madan Samuel\...,2002,471.0,263.0,1170.0,NaN,NaN,NaN,NaN
394,3385,https://www.ncbi.nlm.nih.gov/pubmed/12688626,The Prevention and Incidence of Asthma and\nMi...,2002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
685,10187,NaN,None,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686,3401,NaN,None,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687,2048,NaN,None,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
688,2040,NaN,None,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Make predictions

Current outputs:
- num_male, num_female, num_total, num_male_evidence_span, num_female_evidence_span, num_total_evidence_span
- num_white, num_black, num_latino, num_asian, race_evidence_span

Targets:
- 'participants___male', 'participants___female', 'participants___total'
- 'participants___white', 'participants___black', 'participants___latino', 'participants___asian'

In [3]:
idxs = df['paper___raw_text'].notna()
texts = df[idxs]['paper___raw_text'].tolist()
extractions = extract.extract_nums_df(texts)
for k in extractions.keys():
    df.loc[idxs, k] = extractions[k]

attempting to add dict_keys(['num_male', 'num_female', 'num_total', 'evidence_span_gender'])


538it [00:00, 1173.18it/s]


attempting to add dict_keys(['num_white', 'num_black', 'num_latino', 'num_asian', 'evidence_span_race'])


538it [00:00, 1307.72it/s]

Unterminated string starting at: line 6 column 23 (char 105)


### Evaluate
Evaluates whether each extracted number is within 1 of the human-labeled value

In [4]:
# Apply some postprocessing (convert percentages)
gender_count_processed = df.apply(eval_extraction.process_gender_counts, axis=1)
df["num_male"] = gender_count_processed.apply(lambda x: x[0]).astype(str)
df["num_female"] = gender_count_processed.apply(lambda x: x[1]).astype(str)

print("Total n (with paper text)", idxs.sum())
eval_extraction.compute_metrics(
    df,
    preds_col_to_gt_col_dict={
        "num_male": "participants___male",
        "num_female": "participants___female",
        "num_total": "participants___total",
    },
)

Total n (with paper text) 538


,target,recall,n_labeled,n_predicted_num
0,num_male,191,221,253
1,num_female,191,221,256
2,num_total,219,259,386
